In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.notebook import trange, tqdm, tqdm_notebook
from joblib import Parallel, delayed
import re
import h5py
import tifffile as tiff


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_processed = data_dir / 'processed'
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\Sun Oh experiments\Trail DR5 PPI'


In [4]:
import os
 
os.environ['JDK_HOME'] = "C:\Program Files\Java\jdk-17"
os.environ['JAVA_HOME'] = "C:\Program Files\Java\jdk-17"

os.environ['PATH'] += ';C:\\Program Files\\Java\\jdk-17\\jre\\bin\\server\\;C:\\Program Files\\Java\\jdk-17\\bin\\;C:\\Program Files\\Java\\jdk-17\\bin\\jar.exe'

# Get info

In [22]:
markers_map = {
    'cycle1': {
        1: 'Hoechst', 
        4: 'TRAIL/DR5',
    },
    'cycle2': {
        1: 'Hoechst', 
        4: 'Phalloidin',
    },
}

def get_info(data_raw, marker_dict):
    cells = []
    conditions = []
    types = []

    cycles = []
    channels = []
    markers = []
    paths = [] 
    
    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        if 'after nuclease' in dirpath or 'wrong' in dirpath:
            continue
        
        for name in sorted(filenames):
            if "tif" in name and "sti" in name \
            and 'overlay' not in name \
            and 'Composite' not in name:
                # Get information from image name
                d_split = dirpath.split('\\')

                cell = d_split[-2]
                condition = d_split[-1].split(' ')[1]
                t = d_split[-1].split(' ')[2]

                n_split = name.split('_')
                ch = int(n_split[-1][-5])

                if 'Phall' in dirpath:
                    cycle = 'cycle2'
                else:
                    cycle = 'cycle1'
                try: marker = marker_dict[cycle][ch] 
                except: continue

                cells.append(cell)
                conditions.append(condition)
                types.append(t)

                cycles.append(cycle)
                channels.append(ch)
                markers.append(marker)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Cell": cells,
            "Condition": conditions,
            'FOV': types, 
            "Cycle": cycles,
            "Channels": channels,
            "Markers": markers,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

In [27]:
df_meta_path = data_dir / 'sun_PPI' / 'metadata' / 'info.csv'
df_meta_path.parent.mkdir(parents=True, exist_ok=True)

df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw, markers_map)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Created df


In [28]:
df

,Cell,Condition,FOV,Cycle,Channels,Markers,Path
0,HCC827,ctrl,fixed,cycle2,1,Hoechst,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
1,HCC827,ctrl,fixed,cycle2,4,Phalloidin,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
2,HCC827,negct,fixed,cycle2,1,Hoechst,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
3,HCC827,negct,fixed,cycle2,4,Phalloidin,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
4,HCC827,O,fixed,cycle2,1,Hoechst,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
5,HCC827,O,fixed,cycle2,4,Phalloidin,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
6,HCC827,ctrl,fixed,cycle1,1,Hoechst,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
7,HCC827,ctrl,fixed,cycle1,4,TRAIL/DR5,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
8,HCC827,ctrl,live,cycle1,1,Hoechst,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
9,HCC827,ctrl,live,cycle1,4,TRAIL/DR5,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."


In [29]:
df.groupby(['Cell', 'Condition', 'FOV']).size()

Cell    Condition  FOV  
HCC827  O          fixed    4
                   live     2
        ctrl       fixed    4
                   live     2
        negct      fixed    4
PC9     O          fixed    4
                   live     4
        ctrl       fixed    4
                   live     4
        negct      fixed    4
dtype: int64

# Save HDF5

In [30]:
import h5py

def save_hdf5(
    path: str, name: str, data: np.ndarray, attr_dict=None, mode: str = "a"
) -> None:
    # Read h5 file
    hf = h5py.File(path, mode)
    # Create z_stack_dataset
    if hf.get(name) is None:
        data_shape = data.shape
        data_type = data.dtype
        max_shape = (data_shape[0],) + data_shape[1:]
        dset = hf.create_dataset(
            name,
            shape=data_shape,
            maxshape=max_shape,
            chunks=True,
            dtype=data_type,
            compression="gzip",
        )
        dset[:] = data
        if attr_dict is not None:
            for attr_key, attr_val in attr_dict.items():
                dset.attrs[attr_key] = attr_val
    else:
        print(f"Dataset {name} exists")

    hf.close()

def read_img(path):
    return skimage.io.imread(path, as_gray=True)

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

In [31]:
df_imgs_path = data_dir / 'sun_PPI' / 'metadata' / 'imgs.csv'
df_imgs_path.parent.mkdir(parents=True, exist_ok=True)

temp_path = data_dir  /'sun_PPI' /  'hdf5' / 'raw'
temp_path.mkdir(parents=True, exist_ok=True)

df_exist = df_imgs_path.is_file()


if not df_exist:
    print('Created df')
    
    group = df.groupby(['Cell', 'Condition', 'FOV'])
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        group_cycle = df_group.groupby('Cycle')
        for cycle, df_cycle in group_cycle:
            channels = df_cycle.Channels.to_list()
            markers = df_cycle.Markers.to_list()
            paths = df_cycle.Path.to_numpy()
    
            imgs = joblib_loop(read_img, paths)
            imgs = np.array(imgs)
            info = {"Channels": channels, "Markers": markers}
            
            # hdf5 as Channel -> Z mapping
            save_hdf5(file_path, cycle, imgs, info)
    df_imgs = pd.DataFrame(rows, columns=['Cell', 'Condition', 'FOV', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Created df


  0%|          | 0/10 [00:00<?, ?it/s]

In [32]:
df_imgs

,Cell,Condition,FOV,Path
0,HCC827,O,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
1,HCC827,O,live,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
2,HCC827,ctrl,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
3,HCC827,ctrl,live,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
4,HCC827,negct,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
5,PC9,O,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
6,PC9,O,live,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
7,PC9,ctrl,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
8,PC9,ctrl,live,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
9,PC9,negct,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...


# Save Tiffile

In [34]:
from skimage import exposure, util

def contrast_str(img, n_min=0.1, n_max=100):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img_rescale

def make_imgs_same_dim(imgs):
    # Get max dimensions
    shapes = np.array([img.shape[1:] for img in imgs])
    min_x, min_y = shapes.min(axis=0)
    imgs_cropped = [img[:, :min_x, :min_y] for img in imgs]
    # imgs_cropped[0] = contrast_str(imgs_cropped[0])
    return imgs_cropped


In [35]:
save_path = data_dir /  'sun_PPI' /'imgs' / 'raw'
save_path.mkdir(parents=True, exist_ok=True)

group = df_imgs.groupby(['Cell', 'Condition', 'FOV'])
for name, df_group in group:
    path = df_group.iloc[0].Path
             
    # Read images
    cycles = []
    imgs_all = []
    channels = []
    with h5py.File(path, "r") as f:
        for k in tqdm(f.keys(), total=len(f.keys()), leave=False):
            cycle = k.split('_')[0]
            channel = f[k].attrs['Channels']

            imgs = f[k][:]
            cycles.append(cycle)
            channels.append(channel)
            imgs[0] = contrast_str(imgs[0])
            imgs_all.append(imgs)
    
    imgs_same_shape = make_imgs_same_dim(imgs_all)
    
    for i, imgs in enumerate(imgs_same_shape):
        temp_path = save_path / '_'.join(np.array(name).astype(str))
        temp_path.mkdir(parents=True, exist_ok=True)

        file_name = f'001_{cycles[i]}.tif'
        file_path = temp_path / file_name

        # Write image
        tiff.imwrite(file_path, imgs)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

# Registration 

In [36]:
from ashlar import fileseries, thumbnail,reg
import matplotlib.pyplot as plt
from ashlar.scripts.ashlar import process_axis_flip

In [44]:
# Loop all images
imgs_dir = data_dir / 'sun_PPI' /'imgs' / 'raw'
save_dir = data_dir / 'sun_PPI' /'imgs' / 'registered'
save_dir.mkdir(parents=True, exist_ok=True)

imgs_dir_list = os.listdir(imgs_dir)

for dir_path in tqdm(imgs_dir_list):
    print(len(os.listdir(imgs_dir / dir_path)))


  0%|          | 0/10 [00:00<?, ?it/s]

2
1
2
2
1
2
2
2
2
2


In [45]:
# Loop all images
imgs_dir = data_dir / 'sun_PPI' /'imgs' / 'raw'
save_dir = data_dir / 'sun_PPI' /'imgs' / 'registered'
save_dir.mkdir(parents=True, exist_ok=True)

imgs_dir_list = os.listdir(imgs_dir)

for dir_path in tqdm(imgs_dir_list):
    n_cycle = len(os.listdir(imgs_dir / dir_path))
    # Create reader for each cycle
    readers = []
    for i in range(1, n_cycle+1):
        reader = fileseries.FileSeriesReader(
            str(imgs_dir / dir_path),
            pattern='{series}_'+f'cycle{i}.tif',
            overlap=0.29,
            width=1,
            height=1,
            layout='snake',
            direction='horizontal',
            pixel_size=0.18872, 
        )
        readers.append(reader)
    reader_1 = readers[0]
    
    # Run stitching
    aligner0 = reg.EdgeAligner(reader_1, channel=0, filter_sigma=2, verbose=False,)
    aligner0.run()
    
    # Generate merge image for 1 cycle
    # Parramter
    mosaic_args = {}
    mosaic_args['verbose'] = False

    mosaic = reg.Mosaic(
            aligner0,aligner0.mosaic_shape,**mosaic_args
        )
    writer_class = reg.TiffListWriter
    writer = writer_class(
            [mosaic], str(save_dir / (dir_path + '_cycle1_ch{channel}.ome.tif'))
    )
    writer.run()
    if n_cycle == 1:
        continue
    # Loop through rest of cycles
    aligners = list()
    aligners.append(aligner0)

    for j in range(1, len(readers)):
        aligners.append(
            reg.LayerAligner(readers[j], aligners[0], channel=0, filter_sigma=2, verbose=False)
        )
        aligners[j].run()
        mosaic = reg.Mosaic(
            aligners[j], aligners[0].mosaic_shape,**mosaic_args
        )
        writer = writer_class(
                [mosaic], str(save_dir / (dir_path +'_cycle'+str(j+1)+'_ch{channel}.ome.tif'))
        )
        writer.run()
    

  0%|          | 0/10 [00:00<?, ?it/s]

    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [-4.  6.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [1. 0.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [0. 7.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [-4.  2.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [3. 4.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [1. 2.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [0. 0.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [0. 0.]


# Save data

In [46]:
def get_info(data_raw, marker_dict):
    cells = []
    conditions = []
    fovs = []
    cycles = []
    channels = []
    markers = []
    paths = [] 

    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        for name in sorted(filenames):
            if "tif" in name:
                # Get information from image name
                n_split = name.split('_')
                cell=n_split[0]                       
                cond=n_split[1]
                fov=n_split[2]
                cycle=n_split[3]
                ch = n_split[4][:3]
                try:
                    marker = marker_dict[cycle][ch]
                except:
                    continue 
                
                cells.append(cell)
                conditions.append(cond)
                fovs.append(fov)
                cycles.append(cycle)
                channels.append(ch)
                markers.append(marker)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            'Cell': cells,
            "Condition": conditions,
            "FOV": fovs,
            "Cycle": cycles,
            "Channels": channels,
            "Markers": markers,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

def read_img(path):
    return skimage.io.imread(path, as_gray=True)

def get_min(imgs):
    shapes = np.array([np.array(img.shape) for img in imgs])
    return np.min(shapes, axis=0)

In [47]:
from collections import defaultdict

markers_map_new = defaultdict(dict)
for k,v in markers_map.items():
    for i, (ch,marker) in enumerate(v.items()):
        markers_map_new[k][f'ch{i}'] = marker
        

In [48]:
data_raw = data_dir  / 'sun_PPI' /'imgs' / 'registered'
df_meta_path = data_dir /  'sun_PPI' / 'metadata' / 'info_sti.csv'

try:
    df_meta_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw, markers_map_new)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Folder is already there
Created df


In [49]:
df

,Cell,Condition,FOV,Cycle,Channels,Markers,Path
0,HCC827,O,fixed,cycle1,ch0,Hoechst,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
1,HCC827,O,fixed,cycle1,ch1,TRAIL/DR5,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
2,HCC827,O,fixed,cycle2,ch0,Hoechst,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
3,HCC827,O,fixed,cycle2,ch1,Phalloidin,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
4,HCC827,O,live,cycle1,ch0,Hoechst,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
5,HCC827,O,live,cycle1,ch1,TRAIL/DR5,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
6,HCC827,ctrl,fixed,cycle1,ch0,Hoechst,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
7,HCC827,ctrl,fixed,cycle1,ch1,TRAIL/DR5,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
8,HCC827,ctrl,fixed,cycle2,ch0,Hoechst,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
9,HCC827,ctrl,fixed,cycle2,ch1,Phalloidin,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...


In [50]:
df_imgs_path = data_dir /  'sun_PPI'  /'metadata' / 'imgs_reg.csv'

temp_path =data_dir /   'sun_PPI'  /'hdf5' / 'registered'
try:
    temp_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_imgs_path.is_file()

if not df_exist:
    print('Created df')
    group = df.groupby(['Cell', 'Condition','FOV'])
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        # if file_path.exists():
        #     continue
        
        channels = df_group.Channels.to_list()
        cycles = df_group.Cycle.to_list()
        markers = df_group.Markers.to_list()
        paths = df_group.Path.to_numpy()
            
        imgs = joblib_loop(read_img, paths)
        min_shape = get_min(imgs)
        imgs_cropped = np.array([img[:min_shape[0], :min_shape[1]] for img in imgs])
        info = {"Cycle": cycles, "Channel": channels, "Marker": markers}
            
            # hdf5 as Channel -> Z mapping
        save_hdf5(file_path, 'imgs', imgs_cropped, info)
    df_imgs = pd.DataFrame(rows, columns=['Cell', 'Condition', 'FOV', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Created df


  0%|          | 0/10 [00:00<?, ?it/s]

In [51]:
df_imgs

,Cell,Condition,FOV,Path
0,HCC827,O,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
1,HCC827,O,live,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
2,HCC827,ctrl,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
3,HCC827,ctrl,live,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
4,HCC827,negct,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
5,PC9,O,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
6,PC9,O,live,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
7,PC9,ctrl,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
8,PC9,ctrl,live,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
9,PC9,negct,fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...


# Segmentation 

In [5]:
from skimage.exposure import match_histograms, rescale_intensity
import cv2
from skimage import img_as_ubyte
from skimage import exposure, util, filters, restoration

def random_crop(image, NEW_IMG_HEIGHT, NEW_IMG_WIDTH):
    cropped_image = tf.image.random_crop(
      image, size=[3, NEW_IMG_HEIGHT, NEW_IMG_WIDTH])

    return cropped_image

def make_color_overlay(input_data):
    """Create a color overlay from 2 channel image data
    
    Args:
        input_data: stack of input images
    
    Returns:
        numpy.array: color-adjusted stack of overlays in RGB mode
    """
    RGB_data = np.zeros(input_data.shape[:3] + (3, ), dtype='float32')
    
    # rescale channels to aid plotting
    for img in range(input_data.shape[0]):
        for channel in range(input_data.shape[-1]):
            # get histogram for non-zero pixels
            percentiles = np.percentile(input_data[img, :, :, channel][input_data[img, :, :, channel] > 0],
                                            [0, 100])
            rescaled_intensity = rescale_intensity(input_data[img, :, :, channel],
                                                       in_range=(percentiles[0], percentiles[1]),
                                                       out_range='float32')
            RGB_data[img, :, :, channel + 1] = rescaled_intensity
        
    # create a blank array for red channel
    return RGB_data

def contrast_str(img, n_min=10, n_max=100):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = rescale_intensity(img, in_range=(p2, p98))
    img_rescale = util.img_as_ubyte(img_rescale)
    return img_rescale

In [6]:
df_imgs_path = data_dir /  'sun_PPI'  /'metadata' / 'imgs_reg.csv'
df_imgs = pd.read_csv(df_imgs_path)

In [7]:
for row in tqdm(df_imgs.itertuples(), total=len(df_imgs)):
    path = row.Path
    
    # Read images
    with h5py.File(path, "r") as f:
        
        markers = f['imgs'].attrs['Marker']
        imgs = f['imgs'][:]
    break

  0%|          | 0/10 [00:00<?, ?it/s]

In [8]:
import napari 

napari.view_image(np.stack(imgs), name=markers, channel_axis=0, contrast_limits=[0,1])

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 862.5, 1145.0), zoom=0.5300405561993047, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 1726.0, 1.0), (0.0, 2291.0, 1.0)), current_step=(863, 1145), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Hoechst' at 0x20e701f0e20>, <Image layer 'TRAIL/DR5' at 0x20e67b22530>, <Image layer 'Hoechst [1]' at 0x20e71a8cbe0>, <Image layer 'Phalloidin' at 0x20e71b1ae90>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=Fa

In [26]:
# Define markers to use for cytosolic segmentation
cyto_markers = ['Phalloidin']

# Define folder and create if don't exsit
whole_seg_path = data_dir / 'sun_PPI'  / 'imgs' / 'segmentation'
whole_seg_path.mkdir(parents=True, exist_ok=True)

# Save combined images
for row in df_imgs.itertuples():
    # Read image
    path = row.Path
    with h5py.File(path, "r") as f:
        imgs = f['imgs'][:]
        markers = f['imgs'].attrs['Marker']
        print(markers)
    # Get dapi and cyto imgaes
    indices = np.isin(markers, cyto_markers)
    img_dapi =  imgs[0]
    if indices.any():
        imgs_cyto = imgs[indices,:]
    else:
        imgs_cyto = imgs[[0]]
    # Contrast streching and combine to rgb image
    img_dapi = contrast_str(img_dapi, n_max=99.9)
        
    imgs_cyto_scaled = []
    for img in imgs_cyto:
        imgs_cyto_scaled = [contrast_str(img, n_max=99.9)]
    img_cyto = np.max(np.array(imgs_cyto_scaled), axis=0)
    img_rgb = np.stack([np.zeros(img_dapi.shape),img_cyto, img_dapi], axis=0).astype(np.uint8)
    
    # Crop and save
    file_name = f'{"_".join(row[1:4])}.tif'
    file_path = whole_seg_path / file_name
    tiff.imwrite(file_path, img_rgb)



['Hoechst' 'TRAIL/DR5' 'Hoechst' 'Phalloidin']
['Hoechst' 'TRAIL/DR5']
['Hoechst' 'TRAIL/DR5' 'Hoechst' 'Phalloidin']
['Hoechst' 'TRAIL/DR5']
['Hoechst' 'TRAIL/DR5' 'Hoechst' 'Phalloidin']
['Hoechst' 'TRAIL/DR5' 'Hoechst' 'Phalloidin']
['Hoechst' 'TRAIL/DR5' 'Hoechst' 'Phalloidin']
['Hoechst' 'TRAIL/DR5' 'Hoechst' 'Phalloidin']
['Hoechst' 'TRAIL/DR5' 'Hoechst' 'Phalloidin']
['Hoechst' 'TRAIL/DR5' 'Hoechst' 'Phalloidin']
